In [1]:
# Пример вызова генплана на большой территории 

import geopandas as gpd
import pandas as pd

from gen_planner.python.src.genplanner import GenPlanner
from zoning import func_zones, gen_plan

zones, roads = GenPlanner(gpd.read_file('1.gpkg'), rotation=0).terr2district2zone2block(gen_plan)

In [1]:

m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)

roads.explore(m=m1, column='road_lvl', categorical=True)
m1

# roads.explore(column='road_lvl', categorical=True,tiles='CartoDB positron')


NameError: name 'zones' is not defined

In [5]:
import geopandas as gpd
from shapely import MultiLineString
from shapely.geometry import Point, Polygon, MultiPolygon, LineString
from shapely.ops import polygonize
import pandas as pd

roads_width = {"high speed highway": 60, "regulated highway": 30, "local road": 10}

roads_poly = roads.copy()
roads_poly.geometry = roads_poly.apply(lambda x: x.geometry.buffer(roads_width.get(x.road_lvl,10) / 2), axis=1)


def polygons_to_linestring(geom: Polygon | MultiPolygon):
    def convert_polygon(polygon: Polygon):
        lines = []
        exterior = LineString(polygon.exterior.coords)
        lines.append(exterior)
        interior = [LineString(p.coords) for p in polygon.interiors]
        lines = lines + interior
        return lines

    def convert_multipolygon(polygon: MultiPolygon):
        return MultiLineString(sum([convert_polygon(p) for p in polygon.geoms], []))

    if geom.geom_type == "Polygon":
        return MultiLineString(convert_polygon(geom))
    return convert_multipolygon(geom)


all_data = pd.concat([zones, roads_poly])['geometry']
polygons = polygonize(all_data.apply(polygons_to_linestring).union_all())
enclosures = gpd.GeoSeries(list(polygons), crs=zones.crs)
enclosures_points = gpd.GeoDataFrame(geometry = enclosures.representative_point(), crs=enclosures.crs).reset_index(drop=True)
enclosures_points = enclosures_points[enclosures_points.(roads,align=False)]
joined = gpd.sjoin(enclosures_points, zones, how="inner", predicate="within").reset_index()
joined['geometry'] = enclosures


ValueError: Lengths of inputs do not match. Left: 328830, Right: 12355

In [ ]:
data.explore()

In [ ]:
zones.to_file('zones.gpkg', driver='GPKG')

In [ ]:
# тут вызовы по сценариям по разным территориям, везде basic по умолчанию, но другие лежат в zoning.


from gen_planner.python.src.genplanner import GenPlanner
from zoning import func_zones, terr_zones, basic_scenario
import geopandas as gpd

zones, roads = GenPlanner(gpd.read_file('project Шлиссельбург.geojson')).district2zone2block(basic_scenario)
zones.geometry = zones.geometry.buffer(-10)
roads.geometry = roads.geometry.buffer(10)
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)

In [ ]:
res, roads = GenPlanner(gpd.read_file('Аэродром_лодейнопольское_поселение.geojson')).district2zone2block(basic_scenario)
res.explore(column='func_zone', tiles='CartoDB positron', cmap='Dark2')

In [ ]:
import geopandas as gpd
from gen_planner.python.src.genplanner import GenPlanner
from zoning import func_zones, basic_scenario

res, roads = GenPlanner(gpd.read_file('светогорск.geojson')).district2zone2block(basic_scenario)
res.explore(column='func_zone', tiles='CartoDB positron', cmap='Dark2')

In [ ]:
# тут вызов деления территориальной зоны на кварталы по минимальному размеру терр.зоны
import geopandas as gpd
from gen_planner.python.src.genplanner import GenPlanner
from zoning import terr_zones

res, roads = GenPlanner(gpd.read_file('светогорск.geojson')).zone2block(terr_zones.residential)
res.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2')